# Sprawdzanie dokładności modelu
Plik test_data.csv jest dostępny tylko dla jury. Zawiera dane testowe - jednakowe dla wszystkich drużyn. Jest w takim samym formacie co zbiór, który otrzymaliście wraz z zadaniem, tylko już z czystymi danymi.

In [2]:
import pandas as pd
import numpy as np

In [4]:
test_data = pd.read_csv("test_data.csv")

### Miejsce na twój kod:

In [5]:
!pip install xgboost
!pip install sklearn

  Using cached xgboost-1.3.3-py3-none-win_amd64.whl (95.2 MB)
  Using cached scipy-1.6.2-cp37-cp37m-win_amd64.whl (32.6 MB)
  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.24.1-cp37-cp37m-win_amd64.whl (6.8 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)


In [6]:
from time import time 
import xgboost as xgb
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

def prepare_data(raw_data):
    # Selecting specific columna form dataset
    X_values = raw_data[['fBodyAccJerk-bandsEnergy()-57,64.1', 'fBodyAcc-meanFreq()-Z', 'angle(X,gravityMean)', 
                         'fBodyAcc-bandsEnergy()-1,16', 'fBodyGyro-min()-X', 'tBodyAccMag-arCoeff()1', 
                         'fBodyGyro-maxInds-X', 'fBodyAcc-maxInds-X', 'fBodyAcc-bandsEnergy()-17,24', 
                         'fBodyAccMag-maxInds', 'angle(Y,gravityMean)', 'fBodyAccJerk-meanFreq()-Z', 
                         'fBodyAcc-bandsEnergy()-9,16', 'tBodyGyroMag-entropy()', 'fBodyAcc-bandsEnergy()-1,16.2', 
                         'fBodyAcc-min()-Y', 'fBodyGyro-maxInds-Z', 'tBodyGyro-entropy()-X', 
                         'fBodyBodyGyroJerkMag-maxInds', 'tBodyAcc-correlation()-X,Y', 'tBodyGyro-correlation()-Y,Z', 
                         'fBodyAcc-maxInds-Z', 'fBodyAccJerk-min()-X', 'tGravityAcc-arCoeff()-X,1', 
                         'tGravityAcc-arCoeff()-Z,1', 'fBodyAcc-bandsEnergy()-1,16.1']]
    
    # Performing standardization
    sc = StandardScaler()
    baked_data = sc.fit_transform(X_values)
    return baked_data

def predict(clf, features):
    
    # Starting the clock, making predictions, then stoping the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Printing time results
    print ("Prediction made in: ",end - start)
    
    # Returning predicted labels
    return y_pred

# Preparing dataframe
df = prepare_data(test_data)

# Loading trained model
clf = xgb.XGBClassifier()
clf.load_model('XGB_classifier.model')

# Making predictions
predicted = predict(clf, df)

Prediction made in:  0.03100132942199707


### Sprawdzanie dokładności:
Predicted - zmienna przechowująca przewidzianą aktywność. Poniższa linijka powinna zwrócić wartość 0-1.

In [7]:
np.mean(predicted == test_data.Activity)

0.9953996031030128